# Alinhamento Gobal

In [124]:
"""
Alinhamento global usando valores de match e mismatch
"""
import io

class Mat:
    def __init__(self, rows, cols):                                          #Definir uma matriz 
        self.mat = [[0 for c in range(cols)]
                    for r in range(rows)]
    def numRows (self): return len(self.mat)
    def numCols (self): return len(self.mat[0])
    def __str__(self):
        return '\n'.join(' '.join(str(val) for val in row)
                         for row in self.mat)
    def __repr__(self):
        return str(self)
    def __getitem__ (self, n):
        return self.mat[n]

counter={}
def subst(x,y):                                                        # Caracteres iguais = +4 
  return 4 if x == y else -2                                          # Caracteres diferentes = -2

def alignment_score(s1, s2, g = -4):
  key = f"{s1}:{s2}"
  counter[key] = counter.get(key, 0) + 1
  if not s1 or not s2:
    return max(len(s1), len(s2)) * g

  return max(
      alignment_score(s1[:-1], s2[:-1]) + subst(s1[-1], s2[-1]),
      alignment_score(s1[:-1], s2) + g,
      alignment_score(s1, s2[:-1]) + g)
  



class NW:
  def __init__(self, s1, s2, g = -4):
    self.s1 = s1
    self.s2 = s2
    self.mat = Mat(len(s1) + 1, len(s2) + 1)
    self.tr  = Mat(len(s1) + 1, len(s2) + 1)
    
    for L in range(len(s1)):
      self.mat[L + 1][0] = g * (L + 1)
      self.tr[L + 1][0]  = 'C'

    for C in range(len(s2)):
      self.mat[0][C + 1] = g * (C + 1)
      self.tr[0][C + 1]  = 'E'

    for L, x1 in enumerate(s1):
      for C, x2 in enumerate(s2):
        possiveis = [
            self.mat[L  ][C    ] + subst(x1, x2),   # Diagonal
            self.mat[L+1][C    ] + g,               # Esquerda
            self.mat[L  ][C + 1] + g,               # Cima
        ]
        dirs = "DEC"

        self.mat[L + 1][C + 1] = max(possiveis)
        self.tr[L + 1][C + 1] = dirs[possiveis.index(self.mat[L + 1][C + 1])]

  def rebuild(self):                                          #Matriz T
    L = len(self.s1)
    C = len(self.s2)
    S1 = ""
    S2 = ""
    
    dirs = {
        'D' : (-1, -1),               # Diagonal
        'E' : ( 0, -1),               # Esquerda
        'C' : (-1,  0)                # Cima
    }

    while L > 0 or C > 0:
      DL, DC = dirs[self.tr[L][C]]

      if self.tr[L][C] == "D":
        S1 = self.s1[L - 1] + S1
        S2 = self.s2[C - 1] + S2
      elif self.tr[L][C] == "E":
        S1 = '-' + S1
        S2 = self.s2[C - 1] + S2
      else:
        S1 = self.s1[L - 1] + S1
        S2 = '-' + S2        

      L += DL
      C += DC

    return S1, S2

  def __repr__(self):
    cols = "-" + self.s2
    lins = "-" + self.s1
    with io.StringIO("") as S:
      print("\n Matriz S :")
      print(' ', *cols, sep = '   ', file = S)
      for L, linha in zip(lins, self.mat):
        print(L, *[f'{x:3d}' for x in linha], file = S)

      print('\n Matriz T :', file = S)

      print(' ', *cols, file = S)
      for L, linha in zip(lins, self.tr):
        print(L, *linha, file = S)

      return S.getvalue()


In [126]:
def teste_align():
    align = NW("ACGT","TACCGT")
    print(align)
    print("\n Score do Alinhamento:", alignment_score("ACGT","TACCGT"))
    print("\n Alinhamento ótimo:",*align.rebuild(), sep="\n")  
teste_align()


 Matriz S :
    -   T   A   C   C   G   T
-   0  -4  -8 -12 -16 -20 -24
A  -4  -2   0  -4  -8 -12 -16
C  -8  -6  -4   4   0  -4  -8
G -12 -10  -8   0   2   4   0
T -16  -8 -12  -4  -2   0   8

 Matriz T :
  - T A C C G T
- 0 E E E E E E
A C D D E E E E
C C D D D D E E
G C D D C D D E
T C D D C D D D


 Score do Alinhamento: 8

 Alinhamento ótimo:
-A-CGT
TACCGT



# Alinhamento Local

### Smith Waterman

In [130]:
import io

class Mat:
    def __init__(self, rows, cols):                                          #Definir uma matriz 
        self.mat = [[0 for c in range(cols)]
                    for r in range(rows)]
    def numRows (self): return len(self.mat)
    def numCols (self): return len(self.mat[0])
    def __str__(self):
        return '\n'.join(' '.join(str(val) for val in row)
                         for row in self.mat)
    def __repr__(self):
        return str(self)
    def __getitem__ (self, n):
        return self.mat[n]


def subst(x,y):                                                         
  return 4 if x == y else -2     
                                  
"""
Alinhamento local usando valores de match(+4) e mismatch(-2) e gap penalty(-4)
"""
class SW:
  def __init__(self, s1, s2, g = -4):
    self.s1 = s1
    self.s2 = s2
    self.mat = Mat(len(s1) + 1, len(s2) + 1)
    for L, x1 in enumerate(s1):
      for C, x2 in enumerate(s2):
            self.mat[L + 1][C + 1] = max(
              self.mat[L  ][C    ] + subst(x1, x2),
              self.mat[L+1][C    ] + g,
              self.mat[L  ][C + 1] + g,
              0
          )

  def __repr__(self):
    cols = "-" + self.s2
    lins = "-" + self.s1
    with io.StringIO("") as S:
      print("\n Matriz S :")
      print(' ', *cols, sep = '   ', file = S)
      for L, linha in zip(lins, self.mat):
        print(L, *[f'{x:3d}' for x in linha], file = S)
      
      return S.getvalue() 

In [131]:
def teste_alignSW():
    align = SW("ACGT","TACCGT")
    print(align)
teste_alignSW()


 Matriz S :
    -   T   A   C   C   G   T
-   0   0   0   0   0   0   0
A   0   0   4   0   0   0   0
C   0   0   0   8   4   0   0
G   0   0   0   4   6   8   4
T   0   4   0   0   2   4  12

